In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import UnexpectedAlertPresentException

from bs4 import BeautifulSoup
import time
import csv

In [2]:
#공공보건 포털사이트의 뉴스 기사를 크롤링함
site = 'http://health.chosun.com/list.html?menu=&more_menu=&more_smenu=&nowcode=1&type=&pn=1'

#csv 파일 저장을 위한 f, utf-8을 인코딩으로 사용
f = open('health_chosun.csv', 'w',encoding='utf-8')
#파일 객체를 생성
wr = csv.writer(f)

In [ ]:
row = 1

chrome_driver = 'D:/study/Python/SW/chromedriver.exe'

#chrome으로 driver를 실행
driver = webdriver.Chrome(chrome_driver)

#조건 성립 시간을 위한 wait 객체 설정
wait = WebDriverWait(driver,20)

#driver의 브라우저 크기를 최대화
driver.maximize_window()
#최대화를 하지 않았을 경우 마우스 포인터가 정확히 div를 클릭하지 못하여 크롤링을 못할 수 있음

#웹의 컴포넌트 로딩을 기다리기 위한 implicitly_wait 함수, 로딩이 끝나면 바로 진행된다.
driver.implicitly_wait(3)

#미리 설정해둔 공공보건 포털사이트로 이동한다.
driver.get(site)

#기사의 제목, 내용, 생성날짜를 저장할 변수를 설정한다.
name=""
content=""
date=""

while True:
    time.sleep(1)
    list_num = len(driver.find_elements_by_css_selector("dl.list_item"))
    while True:
        #변수들의 초기화
        name=""
        content=""
        date=""
        while True:
            news_list = driver.find_element_by_css_selector("div#list_area dl:nth-child(" + str(list_num) +")")
            news = news_list.find_element_by_css_selector('dt a')
            news_text = news_list.text
            #잡다한 만화, 책광고 등등의 기사를 제외시킴
            if news.text.find("카툰") != -1 or news.text.find("해랑")!= -1 or news.text.find("서적") != -1 or news.text.find("튼튼선생") != -1 or news.text.find("양냥") != -1 or news_text.find("비만클리닉") != -1:
                print(str(row) + "오류발생")
                list_num -= 1
                if(list_num < 1):
                    break;
            else:
                break;      
        if(list_num < 1):
            break;
        news.click()
        try:
            alert = driver.switch_to.alert
            alert.accept()
            time.sleep(2)
        except :
            pass
        try :
            time.sleep(1)
            name = driver.find_element_by_css_selector("h1#news_title_text_id").text
            date = driver.find_element_by_css_selector("p#date_text").text.split(" ")[1]
            parts = driver.find_element_by_css_selector("div.par")
            content = parts.text
            wr.writerow([row, name, date, content])
            print(str(row) + " " + name)
            row +=1
            driver.back()
            driver.implicitly_wait(5)
            list_num -= 1
            if(list_num < 1):
                break;            
        except (AttributeError, NoSuchElementException, UnexpectedAlertPresentException):
            driver.back()
            list_num -= 1
            if(list_num < 1):
                break;
            
    page_now = driver.find_element_by_css_selector("div.paginate ul.paginate_num span.listAct").text.split(" ")[0]
    url = "http://health.chosun.com/list.html?menu=&more_menu=&more_smenu=&nowcode=1&type=&pn=" + str(int(page_now) -1)
    driver.get(url)